In [2]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import terra
import numpy as np
import meerkat as mk
import pandas as pd

In [3]:
from domino.data.celeb import build_celeb_df, build_celeb_dp

from domino.evaluate.linear import induce_correlation, CorrelationImpossibleError
from meerkat.contrib.celeba import ATTRIBUTES

In [4]:
if False:
    df = build_celeb_df().load()
else:
    df = build_celeb_df.out(504)
dp = build_celeb_dp.out(542, load=True)

In [5]:
dp

,file (NumpyArrayColumn),identity (NumpyArrayColumn),5_o_clock_shadow (NumpyArrayColumn),arched_eyebrows (NumpyArrayColumn),attractive (NumpyArrayColumn),bags_under_eyes (NumpyArrayColumn),bald (NumpyArrayColumn),bangs (NumpyArrayColumn),big_lips (NumpyArrayColumn),big_nose (NumpyArrayColumn),...,wearing_lipstick (NumpyArrayColumn),wearing_necklace (NumpyArrayColumn),wearing_necktie (NumpyArrayColumn),young (NumpyArrayColumn),img_path (NumpyArrayColumn),image_id (NumpyArrayColumn),split (NumpyArrayColumn),index (ListColumn),img (ImageColumn),input (ImageColumn)
0,000001.jpg,2880,0,1,1,0,0,0,0,0,...,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000001.jpg,train,'0',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
1,000002.jpg,2937,0,0,0,1,0,0,0,1,...,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000002.jpg,train,'1',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
2,000003.jpg,8692,0,0,0,0,0,0,1,0,...,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000003.jpg,train,'2',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
3,000004.jpg,5805,0,0,1,0,0,0,0,0,...,1,1,0,1,/home/common/datasets/celeba/img_align_celeba/...,000004.jpg,train,'3',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
4,000005.jpg,9295,0,1,1,0,0,0,1,0,...,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000005.jpg,train,'4',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,9761,0,0,1,0,0,0,1,0,...,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202595.jpg,test,'202594',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
202595,202596.jpg,7192,0,0,0,0,0,1,1,0,...,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202596.jpg,test,'202595',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
202596,202597.jpg,9852,0,0,0,0,0,0,0,0,...,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202597.jpg,test,'202596',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...
202597,202598.jpg,5570,0,1,1,0,0,0,1,0,...,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202598.jpg,test,'202597',ImagePathCell(/home/common/datasets/celeba/img...,ImagePathCell(/home/common/datasets/celeba/img...


In [6]:
from domino.evaluate.linear import induce_correlation, CorrelationImpossibleError, check_corr_induction
from meerkat.contrib.celeba import ATTRIBUTES
check_df = check_corr_induction(df, ATTRIBUTES, corr_start=0, corr_end=0.8, n=3e4, match_mu=False).load()

task: check_corr_induction, run_id=791


In [9]:
check_df = terra.out(623).load()
print(check_df.success.sum())
check_df[check_df.success][50:]

106


,attr_a,attr_b,success
328,blond_hair,narrow_eyes,True
337,blond_hair,straight_hair,True
339,blond_hair,wearing_earrings,True
342,blond_hair,wearing_necklace,True
387,brown_hair,oval_face,True
389,brown_hair,pointy_nose,True
394,brown_hair,straight_hair,True
396,brown_hair,wearing_earrings,True
412,bushy_eyebrows,narrow_eyes,True
429,chubby,double_chin,True


In [20]:
for target, correlate in [
    ("male", "smiling"),
    ("arched_eyebrows", "black_hair"),
    ("eyeglasses", "wearing_necktie"),
    ("wearing_necktie", "eyeglasses"),
    ("mustache", "wearing_hat"),
    ("sideburns", "mustache"),
    ("young", "no_beard"),
    ("receding_hairline", "wearing_necktie"),
    ("bushy_eyebrows", "5_o_clock_shadow"),
    ("wavy_hair", "male"),
    ("wearing_necklace", "blond_hair")
]:
    indices = induce_correlation(
        dp, 
        corr=0.8, 
        attr_a=target, 
        attr_b=correlate, 
        n=30000,
    )
    print('here')

here
here
here
here
here
here
here
here
here
here
here


In [28]:
train_dp = dp.lz[dp["split"].data == "train"]
indices = induce_correlation(
    df, 
    corr=0.85, 
    attr_a="eyeglasses", 
    attr_b="wearing_necktie", 
    n=25000,
)

In [32]:
new_df = df.iloc[indices]
_df = new_df[new_df.split == "train"]

from scipy.stats import pearsonr

pearsonr(_df["eyeglasses"], _df["wearing_necktie"])

(0.8471659195374849, 0.0)

In [6]:
from domino.vision import train


In [ ]:
train(
    dp=dp, #mk.concat([train_dp.lz[indices], dp.lz[dp["split"].data == "valid"]]),
    input_column = "input",
    target_column="smiling",
    id_column="file",
    batch_size=256,
    run_dir="/home/sabri/data/test/1"
)

In [20]:
df

,file,identity,5_o_clock_shadow,arched_eyebrows,attractive,bags_under_eyes,bald,bangs,big_lips,big_nose,...,wavy_hair,wearing_earrings,wearing_hat,wearing_lipstick,wearing_necklace,wearing_necktie,young,img_path,image_id,split
0,000001.jpg,2880,0,1,1,0,0,0,0,0,...,0,1,0,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000001.jpg,train
1,000002.jpg,2937,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000002.jpg,train
2,000003.jpg,8692,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000003.jpg,train
3,000004.jpg,5805,0,0,1,0,0,0,0,0,...,0,1,0,1,1,0,1,/home/common/datasets/celeba/img_align_celeba/...,000004.jpg,train
4,000005.jpg,9295,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,000005.jpg,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,9761,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202595.jpg,test
202595,202596.jpg,7192,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202596.jpg,test
202596,202597.jpg,9852,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202597.jpg,test
202597,202598.jpg,5570,0,1,1,0,0,0,1,0,...,1,1,0,1,0,0,1,/home/common/datasets/celeba/img_align_celeba/...,202598.jpg,test


In [57]:
target, correlate = "high_cheekbones", "wearing_hat"
indices = induce_correlation(
    dp,
    corr=0.85,
    attr_a=target,
    attr_b=correlate,
    n=30000,
    match_mu=True
)

In [58]:
model = terra.get_artifacts(538, "best_chkpt")["model"].load()

/home/common/envs/conda/envs/domino/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [61]:
valid_dp = dp.lz[indices]
valid_dp = valid_dp.lz[valid_dp["split"].data == "valid"]

In [62]:
from domino.vision import predict

In [63]:
valid_dp = predict(
    model,
    valid_dp,
    {"layer4": model.model.layer4},
    batch_size=128
)

In [64]:
from sklearn.metrics import roc_auc_score
target, correlate = "high_cheekbones", "wearing_hat"

In [65]:
roc_auc_score(valid_dp[target], valid_dp["output"].data[:, -1])

0.984703312607204

In [66]:
subgroup_dp = valid_dp[valid_dp[correlate] == 1]
roc_auc_score(subgroup_dp[target], subgroup_dp["output"].data[:, -1])

0.8628345498783454

In [67]:
subgroup_dp = valid_dp[valid_dp[correlate] == 0]
roc_auc_score(subgroup_dp[target], subgroup_dp["output"].data[:, -1])

0.9549199010158895